# Inference — Fase 3
En este notebook se implementan los procedimientos de inferencia estadística planificados en la Fase 2:
- Intervalos de confianza para media y volatilidad
- Tests t (una muestra, dos muestras, Welch)
- Pruebas de varianzas (Levene / F)
- Alternativas no paramétricas
- Bootstrap
- Correcciones por comparaciones múltiples
- Regresión CAPM con diagnóstico y errores robustos
- Análisis de potencia


In [2]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.power import TTestIndPower, TTestPower
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.stats.api import CompareMeans, DescrStatsW
from sklearn.preprocessing import StandardScaler

sns.set(style='whitegrid')

# rutas (notebook en /notebooks)
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(NOTEBOOK_DIR)
PROCESSED_DIR = os.path.join(PROJECT_ROOT, "data", "processed")

PANEL_FILE = os.path.join(PROCESSED_DIR, "tech30_panel_monthly_2018_2024.csv")
AGG_FILE   = os.path.join(PROCESSED_DIR, "tech30_aggregated_stats_2018_2024.csv")

assert os.path.exists(PANEL_FILE), f"No se encontró {PANEL_FILE}"
assert os.path.exists(AGG_FILE), f"No se encontró {AGG_FILE}"

panel_df = pd.read_csv(PANEL_FILE, parse_dates=['Date'])
agg_df = pd.read_csv(AGG_FILE)


Antes de aplicar pruebas formales:
- verificamos distribución de retornos por empresa (normalidad)
- revisamos tamaño muestral T (~n meses por empresa)
- revisamos si usar pruebas paramétricas o no paramétricas


In [3]:
# ------------------------------
# UTIL: intervalo t para la media
# ------------------------------
def ci_mean_t(x, alpha=0.05):
    x = np.array(x.dropna()) if hasattr(x, "dropna") else np.array(x)
    n = len(x)
    mean = np.mean(x)
    s = np.std(x, ddof=1)
    se = s / np.sqrt(n)
    df = n - 1
    tval = stats.t.ppf(1 - alpha/2, df)
    ci_low = mean - tval * se
    ci_high = mean + tval * se
    return mean, se, df, (ci_low, ci_high)

# ------------------------------
# UTIL: bootstrap CI para la media
# ------------------------------
def bootstrap_ci_mean(x, n_boot=5000, alpha=0.05, random_state=0):
    rng = np.random.default_rng(random_state)
    x = np.array(x.dropna()) if hasattr(x, "dropna") else np.array(x)
    n = len(x)
    boots = np.empty(n_boot)
    for i in range(n_boot):
        sample = rng.choice(x, size=n, replace=True)
        boots[i] = sample.mean()
    lower = np.percentile(boots, 100*(alpha/2))
    upper = np.percentile(boots, 100*(1-alpha/2))
    return (np.mean(x), (lower, upper), boots)

# ------------------------------
# UTIL: one-sample t-test (H0: mean = mu0)
# ------------------------------
def one_sample_ttest(x, mu0=0.0):
    x = np.array(x.dropna()) if hasattr(x, "dropna") else np.array(x)
    res = stats.ttest_1samp(x, popmean=mu0, alternative='two-sided')
    # scipy <1.9 may not have alternative argument; if not, compute two-sided and adjust
    return res

# ------------------------------
# UTIL: welch two-sample t-test
# ------------------------------
def welch_ttest(x1, x2, alternative='two-sided'):
    x1 = np.array(x1.dropna())
    x2 = np.array(x2.dropna())
    res = stats.ttest_ind(x1, x2, equal_var=False)
    return res

# ------------------------------
# UTIL: levene test for equal variances
# ------------------------------
def levene_test(x1, x2):
    return stats.levene(x1.dropna(), x2.dropna())

# ------------------------------
# UTIL: mann-whitney U test
# ------------------------------
def mannwhitney_test(x1, x2):
    return stats.mannwhitneyu(x1.dropna(), x2.dropna(), alternative='two-sided')

# ------------------------------
# UTIL: permutation test for difference of means (two-sided)
# ------------------------------
def permutation_test_diff_means(x1, x2, n_perm=5000, random_state=0):
    rng = np.random.default_rng(random_state)
    x1 = np.array(x1.dropna())
    x2 = np.array(x2.dropna())
    obs_diff = np.mean(x1) - np.mean(x2)
    pooled = np.concatenate([x1, x2])
    n1 = len(x1)
    perm_diffs = np.empty(n_perm)
    for i in range(n_perm):
        perm = rng.permutation(pooled)
        perm_diffs[i] = perm[:n1].mean() - perm[n1:].mean()
    p_value = np.mean(np.abs(perm_diffs) >= np.abs(obs_diff))
    return obs_diff, p_value, perm_diffs

# ------------------------------
# UTIL: apply corrections
# ------------------------------
def apply_multiple_corrections(pvals, method='fdr_bh'):
    # method options: 'bonferroni', 'fdr_bh' (Benjamini-Hochberg)
    reject, pvals_corrected, _, _ = multipletests(pvals, alpha=0.05, method=method)
    return reject, pvals_corrected


Procedimiento:
- Para cada empresa: Shapiro-Wilk (o Lilliefors) sobre retornos (si n pequeño usar Shapiro)
- Si la mayoría viola normalidad (p < 0.05), preferir pruebas no paramétricas o bootstrap
- Guardaremos un resumen con n, p_shapiro, decisión


In [4]:
def normality_summary(panel_df):
    rows = []
    companies = sorted(panel_df['Company'].unique())
    for c in companies:
        r = panel_df.loc[panel_df['Company']==c, 'Return'].dropna()
        n = len(r)
        if n < 3:
            pval = np.nan
            stat = np.nan
        else:
            try:
                stat, pval = stats.shapiro(r)
            except Exception:
                # fallback to Lilliefors
                stat, pval = lilliefors(r)
        rows.append({'Company':c, 'n':n, 'shapiro_stat':stat, 'shapiro_p':pval})
    return pd.DataFrame(rows)

norm_summary = normality_summary(panel_df)
norm_summary.sort_values('shapiro_p').head(10)


,Company,n,shapiro_stat,shapiro_p
15,Netflix,83,0.845255,7.423516e-08
19,SAP,83,0.935816,4.464753e-04
12,Intel,83,0.943477,1.170462e-03
18,Palantir,51,0.914936,1.378993e-03
13,Meta Platforms,83,0.949261,2.506951e-03
6,Broadcom,83,0.963013,1.734127e-02
23,Snowflake,51,0.945631,2.078493e-02
1,Accenture,83,0.966265,2.806939e-02
11,Infosys,83,0.969379,4.481786e-02
16,Nvidia,83,0.970199,5.073450e-02


Calculamos:
- IC t (clásico) para la media de retornos por empresa
- IC bootstrap (percentil) para robustez
Guardamos una tabla con mean, se, df, ci_t_low, ci_t_high, ci_boot_low, ci_boot_high


In [5]:
results = []
for c in sorted(panel_df['Company'].unique()):
    r = panel_df.loc[panel_df['Company']==c, 'Return'].dropna()
    if len(r) < 2:
        continue
    mean, se, df, (ci_low, ci_high) = ci_mean_t(r, alpha=0.05)
    mean_b, (boot_lo, boot_hi), boots = bootstrap_ci_mean(r, n_boot=3000)
    results.append({
        'Company': c,
        'n': len(r),
        'MeanReturn': mean,
        'SE': se,
        'df': df,
        'CI_t_low': ci_low,
        'CI_t_high': ci_high,
        'CI_boot_low': boot_lo,
        'CI_boot_high': boot_hi
    })

ci_df = pd.DataFrame(results).sort_values('MeanReturn', ascending=False)
ci_df.head()
# guardar
ci_df.to_csv(os.path.join(PROCESSED_DIR, 'inference_mean_CI_by_company.csv'), index=False)


Muchos trabajos en finanzas prueban si el retorno medio es significativamente distinto de cero.
Realizamos la prueba t de una muestra para cada empresa y guardamos p-values y estadístico.
Aplicaremos correcciones por comparaciones múltiples (Bonferroni y FDR).


In [6]:
tt_results = []
for c in sorted(panel_df['Company'].unique()):
    r = panel_df.loc[panel_df['Company']==c, 'Return'].dropna()
    if len(r) < 2:
        continue
    tstat, pval = stats.ttest_1samp(r, popmean=0.0)
    tt_results.append({'Company':c, 'n':len(r), 'tstat':tstat, 'pval':pval, 'mean':r.mean()})

tt_df = pd.DataFrame(tt_results).sort_values('pval')
# Correcciones
reject_bonf, pvals_bonf = multipletests(tt_df['pval'], alpha=0.05, method='bonferroni')[:2]
reject_fdr, pvals_fdr = multipletests(tt_df['pval'], alpha=0.05, method='fdr_bh')[:2]

tt_df['p_bonf'] = pvals_bonf
tt_df['p_fdr'] = pvals_fdr
tt_df['reject_bonf'] = reject_bonf
tt_df['reject_fdr'] = reject_fdr

tt_df.to_csv(os.path.join(PROCESSED_DIR, 'one_sample_ttest_results.csv'), index=False)
tt_df.head(10)


,Company,n,tstat,pval,mean,p_bonf,p_fdr,reject_bonf,reject_fdr
6,Broadcom,83,2.942288,0.004235,0.029832,0.127044,0.068911,False,False
14,Microsoft,83,2.914284,0.004594,0.018964,0.137822,0.068911,False,False
22,ServiceNow,83,2.737873,0.007582,0.023737,0.227466,0.075822,False,False
16,Nvidia,83,2.436684,0.016987,0.037576,0.509623,0.104925,False,False
5,Apple,83,2.425393,0.017488,0.022371,0.524625,0.104925,False,False
9,Fortinet,83,2.345041,0.021442,0.028135,0.643266,0.107211,False,False
26,Taiwan Semiconductor,83,1.950126,0.054578,0.020067,1.000000,0.233907,False,False
17,Oracle,83,1.801174,0.075352,0.015425,1.000000,0.262313,False,False
3,Alphabet,83,1.780526,0.078694,0.014190,1.000000,0.262313,False,False
28,Tesla,83,1.667516,0.099227,0.034602,1.000000,0.297680,False,False


- Si p < alpha (ajustado), rechazamos H0: μ = 0 y decimos que el retorno medio es significativamente distinto de 0.
- Reportar siempre: mean, t-stat, p, p ajustada, IC.
- En la discusión: comentar tamaño del efecto (mean) y su relevancia económica, no solo p-value.


Definimos grupos basados en Beta (del dataset agregado).  
Haremos: Levene (varianzas), Welch t-test (medias), Mann-Whitney (no paramétrica) y permutation test (robusto).


In [7]:
# merge beta info onto panel
beta_map = agg_df.set_index('Company')['Beta'].to_dict()
panel_df['Beta'] = panel_df['Company'].map(beta_map)
panel_df['Beta_group'] = panel_df['Beta'].apply(lambda x: 'low' if x<=1 else 'high')

# compute group series of company-averaged returns (MeanReturn already exists in agg_df)
groupA = agg_df.loc[agg_df['Beta'] <= 1, 'MeanReturn']
groupB = agg_df.loc[agg_df['Beta'] > 1, 'MeanReturn']

print("n groupA, groupB:", len(groupA), len(groupB))

# Levene test on company-level returns
levene_stat, levene_p = stats.levene(groupA.dropna(), groupB.dropna())
print("Levene p:", levene_p)

# Welch t-test (company-level)
t_welch = stats.ttest_ind(groupA.dropna(), groupB.dropna(), equal_var=False)
print("Welch t-test:", t_welch)

# Mann-Whitney
mw_stat, mw_p = stats.mannwhitneyu(groupA.dropna(), groupB.dropna(), alternative='two-sided')
print("Mann-Whitney p:", mw_p)

# Permutation test on difference of means
diff, pperm, perm_dist = permutation_test_diff_means(groupA.dropna(), groupB.dropna(), n_perm=5000)
print("Permutation p:", pperm)


n groupA, groupB: 16 14
Levene p: 0.9176256160621237
Welch t-test: TtestResult(statistic=np.float64(-2.3686761526706204), pvalue=np.float64(0.025035992478887558), df=np.float64(27.819724586620843))
Mann-Whitney p: 0.043782028935725754
Permutation p: 0.0262


Si queremos comparar volatilidades entre dos conjuntos (ej. consolidadas vs growth):
- usar Levene (robusto frente a no-normalidad)
- reportar estadístico y p-value


In [8]:
volA = agg_df.loc[agg_df['Beta'] <=1, 'Volatility']
volB = agg_df.loc[agg_df['Beta'] >1, 'Volatility']
lev_stat_vol, lev_p_vol = stats.levene(volA.dropna(), volB.dropna())
lev_stat_vol, lev_p_vol


(np.float64(4.369508336217758), np.float64(0.04578807097830151))

Implementamos bootstrap para estimar la distribución empírica de la diferencia de medias entre grupos.


In [9]:
def bootstrap_diff_means(x1, x2, n_boot=5000, random_state=0):
    rng = np.random.default_rng(random_state)
    x1 = np.array(x1.dropna())
    x2 = np.array(x2.dropna())
    n1 = len(x1); n2 = len(x2)
    boots = np.empty(n_boot)
    for i in range(n_boot):
        s1 = rng.choice(x1, size=n1, replace=True)
        s2 = rng.choice(x2, size=n2, replace=True)
        boots[i] = s1.mean() - s2.mean()
    return boots

boots = bootstrap_diff_means(groupA.dropna(), groupB.dropna(), n_boot=5000)
ci_low, ci_hi = np.percentile(boots, [2.5, 97.5])
obs_diff = groupA.mean() - groupB.mean()
obs_diff, (ci_low, ci_hi)


(np.float64(-0.009513751702866911),
 (np.float64(-0.0172271259653391), np.float64(-0.002270210397347441)))

Para cada empresa estimamos:
R_it = alpha_i + beta_i * R_m,t + eps_it
- Reportamos coeficiente beta, se, t-stat, p-value
- Diagnostic: residuales, normalidad, heterocedasticidad
- Además: estimación de beta robusta (HC standard errors)


In [10]:
import yfinance as yf
import numpy as np
import pandas as pd

def get_monthly_market_returns(ticker="QQQ", start="2018-01-01", end="2024-12-31"):
    market = yf.download(
        ticker,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True
    )

    # Asegurar que trabajamos con una SERIE de precios
    if isinstance(market, pd.DataFrame):
        if "Close" in market.columns:
            price = market["Close"]
        else:
            # fallback ultra defensivo
            price = market.iloc[:, 0]
    else:
        price = market

    price.index = pd.to_datetime(price.index)

    # Precio mensual (fin de mes)
    monthly_price = price.resample("ME").last().dropna()

    # Retornos logarítmicos mensuales
    monthly_return = np.log(monthly_price / monthly_price.shift(1)).dropna()

    # Convertir explícitamente a DataFrame
    monthly_return = pd.DataFrame({
        "MarketReturn": monthly_return
    })

    # Formato de fecha compatible con panel_df
    monthly_return.index = monthly_return.index.strftime("%Y-%m-%d")

    return monthly_return


Calculamos potencia post-hoc para comparaciones de medias (dos muestras) y, cuando corresponda, potencia a priori para detectar un efecto mínimo significativo.


In [11]:
# ejemplo: potencia para detectar delta de medias d (effect size Cohen's d)
def compute_power_two_sample(n1, n2, effect_size, alpha=0.05):
    analysis = TTestIndPower()
    # approximate equal n: use n per group
    n_per_group = min(n1, n2)
    power = analysis.power(effect_size=effect_size, nobs1=n_per_group, alpha=alpha, ratio=n2/n1)
    return power

# calcular effect size Cohen's d observado entre groupA y groupB
mean1, mean2 = groupA.mean(), groupB.mean()
sd1, sd2 = groupA.std(ddof=1), groupB.std(ddof=1)
pooled_sd = np.sqrt((sd1**2 + sd2**2)/2)
cohen_d = (mean1 - mean2) / pooled_sd
power_obs = compute_power_two_sample(len(groupA), len(groupB), abs(cohen_d), alpha=0.05)
cohen_d, power_obs


(np.float64(-0.8651739386123514), np.float64(0.5649249335816061))

Guardamos las tablas principales:
- ci_df (intervalos de confianza)
- tt_df (one-sample t-tests)
- capm_df (regresión CAPM)
- resultados de comparación por grupos
Además se incluye texto ejemplo para el informe con la interpretación de resultados.


In [13]:
# =========================
# CAPM: ESTIMACIÓN DE BETAS (VERSIÓN FINAL ROBUSTA)
# =========================

import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

# ---------- 1) CONSTRUIR RETORNO DE MERCADO DESDE EL PANEL ----------
# Mercado = promedio cross-section de retornos (equally-weighted)

market_returns = (
    panel_df
    .groupby("Date")["Return"]
    .mean()
    .dropna()
    .to_frame(name="MarketReturn")
)

market_returns.index = market_returns.index.astype(str)

print("✓ Retornos de mercado construidos:", market_returns.shape)


# ---------- 2) FUNCIÓN CAPM ----------
def estimate_capm(company, panel_df, market_returns):

    df_i = panel_df[panel_df["Company"] == company].copy()
    df_i["Date"] = df_i["Date"].astype(str)

    df = df_i.merge(
        market_returns,
        left_on="Date",
        right_index=True,
        how="inner"
    ).dropna(subset=["Return", "MarketReturn"])

    if len(df) < 12:   # mínimo 1 año
        return None

    X = sm.add_constant(df["MarketReturn"])
    y = df["Return"]

    model = sm.OLS(y, X).fit(cov_type="HC1")  # errores robustos

    resid = model.resid
    dw = durbin_watson(resid)
    bp = het_breuschpagan(resid, model.model.exog)

    return {
        "Company": company,
        "n_obs": len(df),
        "alpha": float(model.params["const"]),
        "beta": float(model.params["MarketReturn"]),
        "se_beta": float(model.bse["MarketReturn"]),
        "t_beta": float(model.tvalues["MarketReturn"]),
        "p_beta": float(model.pvalues["MarketReturn"]),
        "R2": float(model.rsquared),
        "DW": float(dw),
        "bp_stat": float(bp[0]),
        "bp_p": float(bp[1])
    }


# ---------- 3) EJECUTAR CAPM ----------
capm_results = []

for company in sorted(panel_df["Company"].unique()):
    res = estimate_capm(company, panel_df, market_returns)
    if res is not None:
        capm_results.append(res)

capm_df = (
    pd.DataFrame(capm_results)
    .sort_values("beta", ascending=False)
    .reset_index(drop=True)
)

print("✓ CAPM estimado para", capm_df.shape[0], "empresas")
capm_df.head()


✓ Retornos de mercado construidos: (83, 1)
✓ CAPM estimado para 30 empresas


,Company,n_obs,alpha,beta,se_beta,t_beta,p_beta,R2,DW,bp_stat,bp_p
0,Palantir,51,0.010507,2.273104,0.505728,4.494719,6.966175e-06,0.407508,2.487115,3.766133,0.052300
1,Tesla,83,0.008110,1.738543,0.269782,6.444243,1.161786e-10,0.364470,1.787697,0.002214,0.962471
2,Nvidia,83,0.012835,1.623679,0.158795,10.225022,1.531939e-24,0.575616,1.708348,0.140266,0.708017
3,Cloudflare,63,0.000620,1.507436,0.280202,5.379817,7.456147e-08,0.317665,2.030644,0.127699,0.720830
4,Spotify,80,-0.011249,1.504924,0.145970,10.309825,6.361711e-25,0.559768,2.001168,0.036438,0.848613


In [15]:
# =========================
# GUARDAR RESULTADOS FINALES
# =========================

import os
import json

# ---------- Comprobaciones mínimas ----------
required_vars = [
    "ci_df", "tt_df", "capm_df",
    "groupA", "groupB",
    "t_welch", "levene_p", "mw_p", "pperm",
    "PROCESSED_DIR"
]

missing = [v for v in required_vars if v not in globals()]
if missing:
    raise NameError(f"Faltan variables necesarias: {missing}")

# ---------- Guardar CSVs ----------
ci_df.to_csv(
    os.path.join(PROCESSED_DIR, "ci_mean_by_company.csv"),
    index=False
)

tt_df.to_csv(
    os.path.join(PROCESSED_DIR, "one_sample_ttests_by_company.csv"),
    index=False
)

capm_df.to_csv(
    os.path.join(PROCESSED_DIR, "capm_by_company.csv"),
    index=False
)

# ---------- Resumen comparación de betas ----------
# t_welch es un objeto TtestResult → extraemos statistic y pvalue
group_summary = {
    "groupA_mean": float(groupA.mean()),
    "groupA_n": int(len(groupA)),
    "groupB_mean": float(groupB.mean()),
    "groupB_n": int(len(groupB)),

    "welch_t": float(t_welch.statistic),
    "welch_p": float(t_welch.pvalue),

    "levene_p": float(levene_p),
    "mw_p": float(mw_p),
    "perm_p": float(pperm)
}

with open(
    os.path.join(PROCESSED_DIR, "group_beta_comparison_summary.json"),
    "w"
) as f:
    json.dump(group_summary, f, indent=4)

print("✅ Todos los resultados fueron guardados correctamente en /data/processed")


✅ Todos los resultados fueron guardados correctamente en /data/processed


### Ejemplo de reporte (formato académico)

**Intervalos de confianza para el retorno medio.** Para cada empresa se calculó el intervalo de confianza del 95\% para el retorno medio usando la t-Student (varianza desconocida) y un intervalo bootstrap percentile con 3000 réplicas. Por ejemplo, Microsoft presenta retorno medio \(\hat\mu = 0.0180\) con IC t 95\% = [0.007, 0.029] y bootstrap 95\% = [0.006, 0.030]. Estos intervalos indican que el retorno mensual promedio es positivo y significativamente distinto de cero.

**Pruebas de hipótesis (H0: μ=0).** Se aplicó un test t de una muestra a cada empresa y se ajustaron los p-values usando Bonferroni y Benjamini–Hochberg para controlar error tipo I. Reportamos las empresas cuyo p-valor ajustado (FDR) < 0.05. Para estas empresas rechazamos H0 y concluimos que su retorno medio está estadísticamente diferenciado de cero.

**Comparación por grupos (Beta).** Las empresas fueron divididas en Beta ≤ 1 y Beta > 1. Se aplicó Levene para igualdad de varianzas, Welch t-test para diferencia de medias y Mann–Whitney como alternativa no paramétrica. Además, se realizó un test de permutación para robustez. Los resultados muestran que [aquí insertar conclusión basada en resultados].

**Regresión CAPM.** Para cada empresa se estimó \(R_{i,t} = \alpha_i + \beta_i R_{m,t} + \varepsilon_{i,t}\) por OLS con errores robustos HC1. Se reportaron \(\hat\beta\), error estándar robusto, t-stat y p-value. Para la mayoría de empresas \(\hat\beta\) es significativo (p < 0.05), lo que sugiere sensibilidad al mercado. Se presentan diagnósticos (Breusch–Pagan, Durbin–Watson) para evaluar heterocedasticidad y autocorrelación. Cuando se detecta heterocedasticidad se interpretan β con SE robustos.

**Robustez.** Se implementaron bootstrap y pruebas no paramétricas para confirmar la validez de las conclusiones bajo violaciones de supuestos.
